<a href='bokeh.pydata.org'><img src="assets/bokeh_logo.svg" alt="Bokeh logo" width="4%;" align="right"/></a>

<a href='http://www.holoviews.org'><img src="assets/header_logo.png" alt="HoloViews logo" width="20%;" align="left"/></a>
<div style="float:right;"><h2>08. Deploying Bokeh Apps</h2></div>

In the previous sections we discovered how to create static, interactive plots using a ``HoloMap`` and declare dynamic interactivity using ``DynamicMap`` and ``Streams`` within a Jupyter notebook. However, we often want to package our visualization or dashboard as a standalone app, which is where Bokeh Server comes in. 

Bokeh Server allows deploying interactive visualizations and dashboards outside the notebook within your organization or even more widely on the web. In this section we will learn how to deploy a HoloViews-based Bokeh app and how to extend a HoloViews visualization with custom Bokeh models.

In [ ]:
import geoviews as gv
import dask.dataframe as dd
from holoviews.operation.datashader import datashade, aggregate, shade
import holoviews as hv

from holoviews.plotting.util import fire
shade.cmap = fire

hv.extension('bokeh')

## Loading data

As before we will load the taxi dataset as a Dask dataframe:

In [ ]:
usecols = ['tpep_pickup_datetime', 'dropoff_x', 'dropoff_y']
ddf = dd.read_csv('../data/nyc_taxi.csv', parse_dates=['tpep_pickup_datetime'], usecols=usecols)
ddf['hour'] = ddf.tpep_pickup_datetime.dt.hour
ddf = ddf.persist()

## Declare the visualization

We will also once again declare a tile source and datashade the dropoff locations:

In [ ]:
from bokeh.models import WMTSTileSource
url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
wmts = gv.WMTS(WMTSTileSource(url=url))

points = hv.Points(ddf, kdims=['dropoff_x', 'dropoff_y'])
shaded = datashade(points).opts(plot=dict(width=800, height=500, xaxis=None, yaxis=None))

## Deploy app in notebook

When working within the Jupyter notebook we can easily start a server using the HoloViews ``BokehRenderer``. We get a handle on the renderer using the ``hv.renderer`` function. Now we can use the ``app`` method passing it our HoloViews object and setting ``show=True``. If your notebook server is not running on the default port (8888), you also need to define the ``websocket_origin``:

In [ ]:
renderer = hv.renderer('bokeh')
server = renderer.app(wmts * shaded, show=True, websocket_origin='localhost:8888')

#### Excercise 1: Launch a bokeh app using bokeh serve

1. Open http://localhost:8888/edit/notebooks/apps/server_app.py
2. Open http://localhost:8888/terminals/1
3. Now execute:

```
source activate hvtutorial
bokeh serve --show apps/server_app.py
```

## Adding custom functionality

Now let us add some custom functionality to the app we just created. We will declare a ``Stream`` that allows selecting the hour of the day and then declare a DynamicMap that applies the selection.

In [ ]:
from bokeh.models import WMTSTileSource
url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
wmts = gv.WMTS(WMTSTileSource(url=url))

stream = hv.streams.Stream.define('HourSelect', hour=0)()
points = hv.Points(ddf, kdims=['dropoff_x', 'dropoff_y'])
dmap = hv.DynamicMap(lambda hour: points.select(hour=hour), streams=[stream])
shaded = datashade(dmap)

hv.opts('RGB [width=800 height=400 xaxis=None yaxis=None]')

hvobj = wmts * shaded
hvobj

We can now manually trigger updates using the ``stream.event`` method:

In [ ]:
stream.event(hour=6)

In a bokeh server app we can link this stream to another widget, which is exactly what we will do now:

#### Exercise 2: Modify app

1. Open http://localhost:8888/edit/notebooks/apps/player_app.py
2. Copy the HoloViews example above into the appropriate place in the player_app script and note where the ``stream.event`` method is called
2. Open http://localhost:8888/terminals/1
3. Now shut down the server we created in exercise 1 
4. Now execute:

```
bokeh serve --show apps/player_app.py
```

## Adding custom interactivity

Finally we will add another plot to our app, which computes a cross-section at the current hover location:

In [ ]:
from bokeh.models import WMTSTileSource
url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
wmts = gv.WMTS(WMTSTileSource(url=url))

stream = hv.streams.Stream.define('HourSelect', hour=0)()
points = hv.Points(ddf, kdims=['dropoff_x', 'dropoff_y'])
dmap = hv.util.Dynamic(points, operation=lambda obj, hour: obj.select(hour=hour),
                       streams=[stream], link_inputs=True)

# Apply aggregation
aggregated = aggregate(dmap, link_inputs=True)

# Shade the data
shaded = shade(aggregated, link_inputs=True)

# Define PointerX stream, attach to points and declare DynamicMap for cross-section and VLine 
pointer = hv.streams.PointerX(x=ddf.dropoff_x.loc[0].compute().iloc[0], source=points)
section = hv.util.Dynamic(aggregated, operation=lambda obj, x: obj.sample(dropoff_x=x),
                          streams=[pointer], link_inputs=False)
vline = hv.DynamicMap(lambda x: hv.VLine(x), streams=[pointer])

# Define options
hv.opts("RGB [width=800 height=400 xaxis=None yaxis=None] VLine (color='black' line_width=1)")
hv.opts("Curve [width=100 yaxis=None] {+framewise} Layout [shared_axes=False]")

hvobj = (wmts * shaded * vline) << section
hvobj

#### Exercise 3: Add linked stream to app

1. Open http://localhost:8888/edit/notebooks/apps/player_app.py again
2. Edit the app to add the functionality described above
2. Open http://localhost:8888/terminals/1
3. Now shut down the server we created in exercise 2
4. Now execute:

```
bokeh serve --show apps/player_app.py
```


# Onwards

Although the code above is more complex than in previous sections, it's actually providing a huge range of custom types of interactivity, which if implemented in Bokeh alone would have required far more than a notebook cell of code.  Hopefully it is clear that arbitrarily complex collections of visualizations and interactive controls can be built from the components provided by HoloViews, allowing you to make simple analyses very easily and making it practical to make even quite complex apps when needed.  The [user guide](http://holoviews.org/user_guide), [gallery](http://holoviews.org/gallery/index.html), and [reference gallery](http://holoviews.org/reference) should have all the information you need to get started with all this power on your own datasets and tasks.  Good luck!